In [ ]:
#%%

# Install necessary packages
# tabix has to be >= 1.17
# ! pip install pandas zarr scipy numcodecs
# ! apt-get update && apt-get install -y bedtools 
# ! apt-get install gcc
# ! apt-get install make
# ! apt-get install libbz2-dev
# ! apt-get install zlib1g-dev
# ! apt-get install libncurses5-dev 
# ! apt-get install libncursesw5-dev
# ! apt-get install liblzma-dev
# ! cd /usr/bin
# ! wget https://github.com/samtools/htslib/releases/download/1.17/htslib-1.17.tar.bz2
# ! tar -vxjf htslib-1.17.tar.bz2
# ! cd htslib-1.17 && ./configure && make && make install
# ! export PATH="$PATH:/usr/bin/htslib-1.17"
# ! source ~/.profile

In [ ]:
#%%

import os
import subprocess

import numpy as np
import pandas as pd
import zarr
from caesar.io.gencode import Gencode
from pyranges import PyRanges as pr

from get_model.dataset.zarr_dataset import RegionMotifDataset
from get_model.utils import print_shape


def download_motif(motif_url, index_url):
    """
    Download motif data from a URL and its index.

    This function uses the `wget` command-line tool to download the motif file and its index.

    Args:
        motif_url (str): URL to the motif file.
        index_url (str): URL to the index file.
    """
    subprocess.run(['wget', '-O', 'hg38.archetype_motifs.v1.0.bed.gz', motif_url], check=True)
    subprocess.run(['wget', '-O', 'hg38.archetype_motifs.v1.0.bed.gz.tbi', index_url], check=True)

def join_peaks(peak_bed, reference_peaks=None):
    """
    Join peaks from a peak file and a reference peak file.

    This function uses the `bedtools` command-line tool to perform the following operations:
    1. Intersect the peak file with the reference peak file.
    2. Save the final result to a bed file.

    Args:
        peak_bed (str): Path to the peak file.
        reference_peaks (str): Path to the reference peak file.

    Returns:
        str: Path to the output bed file containing the joined peaks.
    """
    if reference_peaks:
        subprocess.run(['bedtools', 'intersect', '-a', peak_bed, '-b', reference_peaks, '-wa', '-u', '-o', 'joint_peaks.bed'], check=True)
    else:
        if os.path.exists('join_peaks.bed'):
            os.remove('join_peaks.bed')
        os.symlink(peak_bed, 'join_peaks.bed')
    return 'join_peaks.bed'

def query_motif(peak_bed, motif_bed):
    """
    Query motif data from a peak file and a motif file.

    This function uses the `tabix` command-line tool to perform the following operations:
    1. Intersect the peak file with the motif file.
    2. Save the final result to a bed file.

    Args:
        peak_bed (str): Path to the peak file.
        motif_bed (str): Path to the motif file.

    Returns:
        str: Path to the output bed file containing the peak motif data.
    """
    subprocess.run(['tabix', motif_bed, '-R', peak_bed], stdout=open('query_motif.bed', 'w'), check=True)
    return 'query_motif.bed'

def get_motif(peak_file, motif_file):
    """
    Get motif data from a peak file and a motif file.

    This function uses the `bedtools` command-line tool to perform the following operations:
    1. Intersect the peak file with the motif file.
    2. Group the intersected data by peak and motif.
    3. Sum the scores for overlapping peaks and motifs.
    4. Sort the resulting data by chromosome, start, end, and motif.
    5. Save the final result to a bed file.

    Args:
        peak_file (str): Path to the peak file.
        motif_file (str): Path to the motif file.

    Returns:
        str: Path to the output bed file containing the peak motif data.
    """

    cmd = f"""
    ASSEMBLY="hg38"
    MEM="12G"
    CPU="2"
    ATAC_PEAK_FILE="{peak_file}"
    ATAC_MOTIF_FILE="{motif_file}"
    OUTPUT_PATH="get_motif.bed"

    OUTPUT_BASE=$(basename "$OUTPUT_PATH" .bed)

    awk -v OUTPUT_BASE="$OUTPUT_BASE" '{{OFS="\\t"; print $1,$2,$3 > OUTPUT_BASE"."$1".peak.bed"}}' "$ATAC_PEAK_FILE"
    awk -v OUTPUT_BASE="$OUTPUT_BASE" '{{print > OUTPUT_BASE"."$1".motif.bed"}}' "$ATAC_MOTIF_FILE"

    for CHR in $(ls "${{OUTPUT_BASE}}."*".motif.bed" | grep -v random | grep -v chrY | cut -d'.' -f2); do
        bedtools intersect -a "${{OUTPUT_BASE}}.${{CHR}}.peak.bed" -b "${{OUTPUT_BASE}}.${{CHR}}.motif.bed" -wa -wb | cut -f1,2,3,7,8,10 > "${{OUTPUT_BASE}}.${{CHR}}.peak_motif.bed"
        sort -k1,1 -k2,2n -k3,3n -k4,4 "${{OUTPUT_BASE}}.${{CHR}}.peak_motif.bed" -o "${{OUTPUT_BASE}}.${{CHR}}.peak_motif.bed"
        bedtools groupby -i "${{OUTPUT_BASE}}.${{CHR}}.peak_motif.bed" -g 1-4 -c 5 -o sum > "${{OUTPUT_BASE}}.${{CHR}}.peak_motif.bed.tmp"
        mv "${{OUTPUT_BASE}}.${{CHR}}.peak_motif.bed.tmp" "${{OUTPUT_BASE}}.${{CHR}}.peak_motif.bed"
        sort -k1,1V -k2,2n -k3,3n -k4,4 "${{OUTPUT_BASE}}.${{CHR}}.peak_motif.bed" -o "${{OUTPUT_BASE}}.${{CHR}}.peak_motif.bed"
    done

    cat $(ls "${{OUTPUT_BASE}}."*".peak_motif.bed" | grep -v random | grep -v chrY | sort -k1,1V) > "$OUTPUT_PATH"

    # Clean up temporary files
    rm "${{OUTPUT_BASE}}."*".peak.bed" "${{OUTPUT_BASE}}."*".motif.bed" "${{OUTPUT_BASE}}."*".peak_motif.bed"

    echo "Peak motif extraction completed. Results saved in $OUTPUT_PATH"
    """
    subprocess.run(cmd, shell=True, check=True)
    return 'get_motif.bed'

def create_peak_motif(peak_motif_bed, output_zarr):
    """
    Create a peak motif zarr file from a peak motif bed file.
    
    This function reads a peak motif bed file, pivots the data, and saves it to a zarr file.
    The zarr file contains three datasets: 'data', 'peak_names', 'motif_names', and 'accessibility'.
    The 'data' dataset is a sparse matrix containing the peak motif data.
    The 'peak_names' dataset contains the peak names.
    The 'motif_names' dataset contains the motif names.
    
    Args:
        peak_motif_bed (str): Path to the peak motif bed file.
        output_zarr (str): Path to the output zarr file.
    """
    # Read the peak motif bed file
    peak_motif = pd.read_csv(peak_motif_bed, sep='\t', header=None, 
                             names=['Chromosome', 'Start', 'End', 'Motif_cluster', 'Score'])
    
    # Pivot the data
    peak_motif_pivoted = peak_motif.pivot_table(index=['Chromosome', 'Start', 'End'], 
                                                columns='Motif_cluster', 
                                                values='Score', 
                                                fill_value=0)
    peak_motif_pivoted.reset_index(inplace=True)
    
    # Create the 'Name' column
    peak_motif_pivoted['Name'] = peak_motif_pivoted.apply(lambda x: f'{x["Chromosome"]}:{x["Start"]}-{x["End"]}', axis=1)
    peak_motif_pivoted = peak_motif_pivoted.drop(columns=['Chromosome', 'Start', 'End'])
    # Read the original peak bed file
    original_peaks = pd.read_csv(peak_bed, sep='\t', header=None, 
                                 names=['Chromosome', 'Start', 'End', 'Score'])
    original_peaks['Name'] = original_peaks.apply(lambda x: f'{x["Chromosome"]}:{x["Start"]}-{x["End"]}', axis=1)
    
    # Merge the pivoted data with the original peaks
    merged_data = pd.merge(original_peaks, peak_motif_pivoted, on='Name', how='left')
    
    # Fill NaN values with 0 for motif columns
    motif_columns = [col for col in merged_data.columns if col not in ['Chromosome', 'Start', 'End', 'Score', 'Name']]
    merged_data[motif_columns] = merged_data[motif_columns].fillna(0)
    # Prepare data for zarr storage
    name_values = list(merged_data['Name'].values)
    motif_values = motif_columns
    
    # Create sparse matrix
    sparse_matrix = merged_data[motif_columns].values
    # Open zarr store and save data
    from numcodecs import Blosc
    z = zarr.open(output_zarr, mode='w')
    z.create_dataset('data', data=sparse_matrix.data, chunks=(1000, sparse_matrix.shape[1]), dtype=np.float32, compressor=Blosc(cname='zstd', clevel=3, shuffle=Blosc.BITSHUFFLE), shape=sparse_matrix.shape)
    z.create_dataset('peak_names', data=name_values)
    z.create_dataset('motif_names', data=motif_values)

    print(f"Peak motif data saved to {output_zarr}")

def zip_zarr(zarr_file):
    subprocess.run(['zip', '-r', f'{zarr_file}.zip', zarr_file], check=True)

def add_atpm(zarr_file, bed_file, celltype):
    """
    Add aTPM (ATAC-seq 'Transcript'/Count Per Million) data for a specific cell type to the zarr file.

    This function reads aTPM data from a BED file and adds it to the zarr file under the 'atpm' group.
    The aTPM values are associated with peak names in the zarr file.

    Args:
        zarr_file (str): Path to the zarr file.
        bed_file (str): Path to the BED file containing aTPM data.
        celltype (str): Name of the cell type for which the aTPM data is being added.

    The BED file should have the following columns:
    1. Chromosome
    2. Start
    3. End
    4. aTPM value

    The function creates an 'atpm' group in the zarr file if it doesn't exist and adds a dataset
    for the specified cell type under this group.
    """
    df = pd.read_csv(
        bed_file,
        sep='\t',
        header=None,
        names=['Chromosome', 'Start', 'End', 'aTPM']
    )
    df['Name'] = df.apply(lambda x: f'{x["Chromosome"]}:{x["Start"]}-{x["End"]}', axis=1)
    name_to_atpm = dict(zip(df['Name'], df['aTPM']))
    z = zarr.open(zarr_file, mode='a')
    
    # Create the atpm group if it doesn't exist
    if 'atpm' not in z:
        z.create_group('atpm')
    
    # Save aTPM data for the specific cell type
    z['atpm'].create_dataset(celltype, data=np.array([name_to_atpm[name] for name in z['peak_names']]), overwrite=True, chunks=(1000,), dtype=np.float32)

def add_exp(zarr_file, rna_file, atac_file, celltype, assembly='hg38', version=40, extend_bp=100):
    """
    Add expression and TSS data for a specific cell type to the zarr file.
    """
    # Initialize Gencode
    gencode = Gencode(assembly=assembly, version=version)
    
    # Read RNA data
    gene_exp = pd.read_csv(rna_file)
    gene_exp['gene_id'] = gene_exp['gene_id'].apply(lambda x: x.split(".")[0])
    promoter_exp = pd.merge(gencode.gtf, gene_exp, left_on='gene_id', right_on='gene_id')
    
    # Read ATAC data
    if atac_file.endswith(".bed"):
        atac = pr(pd.read_csv(atac_file, sep='\t', header=None, 
                              names=['Chromosome', 'Start', 'End', 'aTPM']).reset_index(), int64=True)
    else:
        atac = pr(pd.read_csv(atac_file, index_col=0).reset_index(), int64=True)
    
    # Join ATAC and RNA data
    exp = atac.join(pr(promoter_exp, int64=True).extend(extend_bp), how='left').as_df()
    
    # Process expression data
    exp = exp[['index', 'Strand', 'TPM']].groupby(['index', 'Strand']).mean().reset_index()
    
    # Calculate expression and TSS
    exp_n = exp[exp.Strand == "-"].set_index('index')['TPM'].fillna(0)
    exp_p = exp[exp.Strand == "+"].set_index('index')['TPM'].fillna(0)
    exp_n[exp_n < 0] = 0
    exp_p[exp_p < 0] = 0
    
    exp_n_tss = (exp[exp.Strand == "-"].set_index('index')['TPM'] >= 0).fillna(False)
    exp_p_tss = (exp[exp.Strand == "+"].set_index('index')['TPM'] >= 0).fillna(False)
    
    tss = np.stack([exp_p_tss, exp_n_tss]).T
    exp_data = np.stack([exp_p, exp_n]).T
    
    # Open zarr file
    z = zarr.open(zarr_file, mode='a')
    
    # Create groups if they don't exist
    for group in ['expression_positive', 'expression_negative', 'tss']:
        if group not in z:
            z.create_group(group)
    
    # Save data for the specific cell type
    peak_names = z['peak_names'][:]
    z['expression_positive'].create_dataset(celltype, data=exp_data[:, 0].astype(np.float32), overwrite=True, chunks=(1000,), dtype=np.float32)
    z['expression_negative'].create_dataset(celltype, data=exp_data[:, 1].astype(np.float32), overwrite=True, chunks=(1000,), dtype=np.float32)
    z['tss'].create_dataset(celltype, data=tss.astype(np.int8), overwrite=True, chunks=(1000,), dtype=np.int8)

In [ ]:
#%%

motif_bed_url='https://resources.altius.org/~jvierstra/projects/motif-clustering/releases/v1.0/hg38.archetype_motifs.v1.0.bed.gz'
motif_bed_index_url='https://resources.altius.org/~jvierstra/projects/motif-clustering/releases/v1.0/hg38.archetype_motifs.v1.0.bed.gz.tbi'
peak_bed='/home/xf2217/Projects/pretrain_human_bingren_shendure_apr2023/fetal_adult/118.atac.bed'
reference_peaks=None

if motif_bed_url and motif_bed_index_url and not (os.path.exists('hg38.archetype_motifs.v1.0.bed.gz') or os.path.exists('hg38.archetype_motifs.v1.0.bed.gz.tbi')):
    download_motif(motif_bed_url, motif_bed_index_url)
    motif_bed = 'hg38.archetype_motifs.v1.0.bed.gz'
else:
    motif_bed = '/home/xf2217/Projects/get_data/hg38.archetype_motifs.v1.0.bed.gz'

In [ ]:
#%%

# join peaks
joint_peaks = join_peaks(peak_bed, reference_peaks)

In [ ]:
#%%

# query motif
peaks_motif = query_motif(joint_peaks, motif_bed)
get_motif_output = get_motif(joint_peaks, peaks_motif)

Peak motif extraction completed. Results saved in get_motif.bed


In [ ]:
#%%

# create peak motif zarr file
create_peak_motif(get_motif_output, 'output.zarr')

Peak motif data saved to output.zarr


In [ ]:
#%%

# add aTPM data for multiple cell types
add_atpm('output.zarr', '/home/xf2217/Projects/pretrain_human_bingren_shendure_apr2023/fetal_adult/118.atac.bed', 'astrocyte')

In [ ]:
#%%

# add expression and TSS data for multiple cell types
add_exp('output.zarr', '/home/xf2217/Projects/pretrain_human_bingren_shendure_apr2023/fetal_adult/118.rna.csv', '/home/xf2217/Projects/pretrain_human_bingren_shendure_apr2023/fetal_adult/118.atac.bed', 'astrocyte')

join: Strand data from other will be added as strand data to self.
If this is undesired use the flag apply_strand_suffix=False.
To turn off the warning set apply_strand_suffix to True or False.
<ipython-input-2-517f65786835>:237: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  exp = exp[['index', 'Strand', 'TPM']].groupby(['index', 'Strand']).mean().reset_index()


In [ ]:
#%%

# optionally zip the zarr file for download or storage
zip_zarr('output.zarr')

updating: output.zarr/ (stored 0%)
updating: output.zarr/motif_names/ (stored 0%)
updating: output.zarr/motif_names/0 (deflated 29%)
updating: output.zarr/motif_names/.zarray (deflated 47%)
updating: output.zarr/expression_positive/ (stored 0%)
updating: output.zarr/expression_positive/astrocyte/ (stored 0%)
updating: output.zarr/expression_positive/astrocyte/77 (deflated 37%)
updating: output.zarr/expression_positive/astrocyte/28 (deflated 34%)
updating: output.zarr/expression_positive/astrocyte/96 (deflated 32%)
updating: output.zarr/expression_positive/astrocyte/73 (deflated 36%)
updating: output.zarr/expression_positive/astrocyte/51 (deflated 36%)
updating: output.zarr/expression_positive/astrocyte/99 (deflated 32%)
updating: output.zarr/expression_positive/astrocyte/41 (deflated 35%)
updating: output.zarr/expression_positive/astrocyte/22 (deflated 33%)
updating: output.zarr/expression_positive/astrocyte/20 (deflated 33%)
updating: output.zarr/expression_positive/astrocyte/108 (def

In [ ]:
#%%

# clean up intermediate files
for file in [joint_peaks, peaks_motif, get_motif_output]:
    os.remove(file)

In [ ]:
# %%

# load the zarr file as a dataset
region_motif_dataset = RegionMotifDataset(
    './output.zarr',
    celltypes=['astrocyte'],
    quantitative_atac=True,
    num_region_per_sample=200,
    is_train=True
)

100%|██████████| 1/1 [00:00<00:00,  3.35it/s]


In [ ]:
# %%

print_shape(region_motif_dataset[0])

region_motif: (200, 283)
mask: (200, 2)
atpm: (200, 1)
chromosome: chr1
peak_coord: (200, 2)
exp_label: (200, 2)
celltype: astrocyte
